In [2]:
import pandas as pd

In [3]:
df = pd.read_csv(r"C:\Users\ayush\Desktop\JUPYTER\DATASET\spotify_millsongdata.csv")
df.head(4)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...


In [4]:
df.shape

(57650, 4)

In [5]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [6]:
df = df.sample(5000).drop("link",axis=1).reset_index(drop=True)
df.head(10)

,artist,song,text
0,Linda Ronstadt,All The Beautiful Things,The rain doesn't fall on me any more \r\nThe ...
1,W.A.S.P.,My Wicked Heart,Am I free \r\nAm I done believing \r\nOh God...
2,Primus,My Friend Fats,"My friend Fats, he's a hell of a guy. \r\nLet..."
3,Johnny Cash,Heavy Metal (Don't Mean Rock And Roll To Me),Some days I think this old machine is out to g...
4,Miley Cyrus,1 In Amillon,How did I get here? I turned around and there ...
5,America,Today's The Day,Hold me close \r\nYou turn nighttime into day...
6,Europe,Spirit Of The Underdog,Seen you round I know your face \r\nWatched y...
7,Foreigner,Lonely Children,"You go where the wind blows, leading the life ..."
8,Morrissey,The World Is Full Of Crashing Bores,"You must be wondering how, \r\nThe boy next d..."
9,Norah Jones,Everybody Needs A Best Friend,My words are lazy \r\nMy thoughts are hazy \...


In [7]:
df["text"][0]

"The rain doesn't fall on me any more  \r\nThe sun doesn't feel as warm as before  \r\nStars seem far away and fading from sight  \r\nAll the beautiful things in my life  \r\nThey're all gone, all gone, all gone now  \r\n  \r\nClose talks, long walks taken with friends  \r\nGetting dirty playing with the neighborhood kids  \r\nWaking up early to see the sun rise  \r\nAll the meaningful things in my life  \r\nThey're all gone, all gone, all gone now  \r\n  \r\nSong birds, summer words in my ear  \r\nThe fingers of the wind brushing my hair  \r\nThe mold on the leaves as they die each year  \r\nThe magical things in my life  \r\nThey're all gone, all gone, all gone now\r\n\r\n"

In [8]:
df.shape

(5000, 3)

# Text cleaning/ Text preprocessing

In [9]:
df['text']=df['text'].str.lower().replace(r'^\w\s',' ').replace(r'\n',' ', regex = True)

In [10]:
import nltk
from nltk.stem.porter import PorterStemmer

In [11]:
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ayush\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\ayush\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [12]:
stemmer = PorterStemmer()

In [13]:
def token(txt):
    token = nltk.word_tokenize(txt)
    a = [stemmer.stem(w) for w in token]
    return " ".join(a)

In [14]:
df['text'].apply(lambda x : token(x))

0       the rain doe n't fall on me ani more the sun d...
1       am i free am i done believ oh god jehovah i 'v...
2       my friend fat , he 's a hell of a guy . let me...
3       some day i think thi old machin is out to get ...
4       how did i get here ? i turn around and there y...
                              ...                        
4995    oh listen , sister i love my mister man and i ...
4996    it 's alright it 's ok i 'll keep hold on toda...
4997    we go down to a foriegn land dive through the ...
4998    darl , je vou aim beaucoup je ne sai pa what y...
4999    no matter who you are no matter what you do th...
Name: text, Length: 5000, dtype: object

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [16]:
tfid = TfidfVectorizer(analyzer='word', stop_words='english')

In [17]:
matrix = tfid.fit_transform(df['text'])

In [18]:
similar = cosine_similarity(matrix)

In [19]:
similar[0]

array([1.        , 0.00424448, 0.00175522, ..., 0.03207341, 0.01728133,
       0.01700973], shape=(5000,))

In [21]:
df[df['song']=='The World Is Full Of Crashing Bores'].index[0]

np.int64(8)

# Recommender Function

In [22]:
def recommender(song_name):
    idx = df[df['song']==song_name].index[0]
    diatance = sorted(list(enumerate(similar[idx])), reverse=True, key=lambda x:x[1])
    song = []
    for s_id in diatance[1:5]:
        song.append(df.iloc[s_id[0]].song)
    return song

In [23]:
recommender('The World Is Full Of Crashing Bores')

['Let Me Take You In My Arms Again',
 'In My Arms',
 'Our Love',
 'I Love You, Earth']

In [24]:
import pickle

In [26]:
pickle.dump(similar,open("similarity", "wb"))

In [27]:
pickle.dump(df,open("df", "wb"))